In [24]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary
import matplotlib.pyplot as plt

from generate_data import generate_hiv_data, generate_discrete_hiv_data
from decimal import Decimal

In [25]:
T = 1e-1
t = np.linspace(0, 100, 1001)
x0 = [1, 1, 1, 1, 1]
x0_val = [2, 1, 2, 2, 3]

u = lambda t: 0.1*np.sin(t)
u_val = lambda t: 0.1*np.cos(2*t)

x, x_dot = generate_hiv_data(t=t, x0=x0, u=u)
x_val, _ = generate_hiv_data(t=t, x0=x0_val, u=u_val)

xk = generate_discrete_hiv_data(t=t, x0=x0, T=T, u=u(t))
xk_val = generate_discrete_hiv_data(t=t, x0=x0_val, T=T, u=u_val(t))

In [26]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=0.00001), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(len(x[0]))]+['u'],
    discrete_time=True)
model.fit(x=xk, u=u(t))
model.print()

(x1)[k+1] = 0.100 1 + 0.990 x1[k] + -0.100 x1[k] x2[k] + 0.098 x1[k] x2[k] u[k]
(x2)[k+1] = 0.980 x2[k] + 0.100 x1[k] x2[k] + -0.100 x2[k] x4[k] + -0.100 x2[k] x5[k] + -0.098 x1[k] x2[k] u[k]
(x3)[k+1] = 0.999 x3[k] + -0.003 x2[k] x3[k] + 0.006 x1[k] x2[k] x3[k]
(x4)[k+1] = 0.990 x4[k] + 0.003 x2[k] x4[k]
(x5)[k+1] = 0.990 x5[k] + 0.003 x2[k] x3[k]


In [27]:
for x_num in range(len(x0)):
    q = 'Biblioteka funkcji & Próg & $\Dot{x}'
    print(f'{q}_{x_num+1}$ & $E_{x_num+1}$ \\\\')
    for i, library in enumerate([FourierLibrary(n_frequencies=2), FourierLibrary(n_frequencies=1), PolynomialLibrary(degree=3), PolynomialLibrary(degree=2), PolynomialLibrary(degree=1)]):
        print(f'\\hline')
        for threshold in range(3):
            threshold = 10**(threshold-4) if i >= 2 else 100*10**(threshold-4)
            threshold = T * threshold
            name = ['Trygonometryczna (st. 2)', 'Trygonometryczna (st. 1)', 'Wielomiany (st. 3)', 'Wielomiany (st. 2)', 'Liniowa']
            model = ps.SINDy(
                feature_library=library,
                optimizer=ps.STLSQ(threshold=threshold),
                feature_names=[f'x{i+1}' for i in range(len(x0))]+['u'],
                discrete_time=True
                )
            model.fit(x=xk, u=u(t))
            try:
                x_sim = model.simulate(x0=x0_val, t=1001, u=u_val(t))
                mse = ((x_sim - xk_val)**2).mean(axis=0)
                E = '%.3E' % Decimal(str(mse[x_num]))
            except:
                E = '\infty'
            coeffs = ' + '.join(['%.3E' % Decimal(str(coeff))+' '+model.get_feature_names()[i] for i, coeff in enumerate(model.coefficients()[x_num]) if abs(model.coefficients()[x_num][i]) > threshold])
            if len(coeffs.split(' + ')) > 2:
                eq = (coeffs.split(' + ')[0] + ' + ' + coeffs.split(' + ')[1] + '\dots').replace(' 1 +', ' +')
            else:
                eq = coeffs
            if len(coeffs) == 0:
                eq = '0,000'
            eq = eq.replace(' 1 +', ' +').replace('.', ',').replace('+ -', '- ').replace('sin', '\sin').replace('cos', '\cos').replace('(1 x1)', '(x_1)').replace('(1 x2)', '(x_2)').replace('x2', 'x_2').replace('(1 x3)', '(x_3)').replace('x3', 'x_3').replace('(1 x4)', '(x_4)').replace('x4', 'x_4').replace('(1 x5)', '(x_5)').replace('x5', 'x_5').replace('x1', 'x_1').replace('(1 u)', '(u)')
            for pow in range(1, 10):
                eq = eq.replace('E+00', '')
                E = E.replace('E+00', '')
                eq = eq.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
                E = E.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
            print(f"{name[i]} & {(str(threshold).replace('.', ',') + ' &').replace(',0 &', ' &')} ${eq}$ & ${E.replace('.', ',')}$ \\\\")
    print('\n\n')

Biblioteka funkcji & Próg & $\Dot{x}_1$ & $E_1$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $-1,354\cdot 10^{-1} \sin(x_1) - 1,322 \cos(x_1)\dots$ & $7,515$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $-3,654\cdot 10^{-2} \sin(x_1) - 1,336 \cos(x_1)\dots$ & $7,350\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,1 & $-1,334 \cos(x_1) - 2,387 \sin(x_2)\dots$ & $8,241\cdot 10^{-1}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $-3,093\cdot 10^{-3} \sin(x_1) - 1,023 \cos(x_1)\dots$ & $9,072\cdot 10^{-1}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $-1,020 \cos(x_1) - 7,053\cdot 10^{-2} \sin(x_2)\dots$ & $5,294\cdot 10^{-1}$ \\
Trygonometryczna (st. 1) & 0,1 & $-1,001 \cos(x_1) + 2,530\cdot 10^{-1} \cos(x_2)\dots$ & $9,495\cdot 10^{-1}$ \\
\hline
Wielomiany (st. 3) & 1e-05 & $1,000\cdot 10^{-1} + 9,900\cdot 10^{-1} x_1\dots$ & $1,082E-10$ \\
Wielomiany (st. 3) & 0,0001 & $1,000\cdot 10^{-1} + 9,900\cdot 10^{-1} x_1\dots$ & $9,832E-12$ \\
Wielomiany (st. 3) & 0,001 & $1,000\cdot

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: over

Liniowa & 1e-05 & $2,624\cdot 10^{-1} + 9,198\cdot 10^{-1} x_1\dots$ & $2,636\cdot 10^{-1}$ \\
Liniowa & 0,0001 & $2,624\cdot 10^{-1} + 9,198\cdot 10^{-1} x_1\dots$ & $2,636\cdot 10^{-1}$ \\
Liniowa & 0,001 & $2,624\cdot 10^{-1} + 9,198\cdot 10^{-1} x_1\dots$ & $3,492\cdot 10^{-1}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_2$ & $E_2$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $2,058\cdot 10^{-1} \sin(x_1) - 2,823\cdot 10^{-2} \cos(x_1)\dots$ & $1,439\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $1,264\cdot 10^{-1} \sin(x_1) - 1,114\cdot 10^{-1} \cos(x_1)\dots$ & $1,091\cdot 10^2$ \\
Trygonometryczna (st. 2) & 0,1 & $6,967\cdot 10^{-1} \sin(x_2) - 6,941\cdot 10^{-1} \cos(x_2)\dots$ & $2,702\cdot 10^{-1}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $6,788\cdot 10^{-2} \sin(x_1) - 4,740\cdot 10^{-2} \cos(x_1)\dots$ & $2,132\cdot 10^{-1}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $7,715\cdot 10^{-2} \sin(x_1) - 5,472\cdot 10^{-2} \cos(x_1)\dots$ & $8,183\c

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: over

Liniowa & 1e-05 & $-9,099\cdot 10^{-3} + 5,833\cdot 10^{-2} x_1\dots$ & $6,653\cdot 10^{-1}$ \\
Liniowa & 0,0001 & $-9,099\cdot 10^{-3} + 5,833\cdot 10^{-2} x_1\dots$ & $6,653\cdot 10^{-1}$ \\
Liniowa & 0,001 & $-4,305\cdot 10^{-3} + 5,783\cdot 10^{-2} x_1\dots$ & $7,196\cdot 10^{-1}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_3$ & $E_3$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $5,514\cdot 10^{-2} \sin(x_1) + 3,706\cdot 10^{-2} \cos(x_1)\dots$ & $3,106\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $5,514\cdot 10^{-2} \sin(x_1) + 3,706\cdot 10^{-2} \cos(x_1)\dots$ & $2,022\cdot 10^4$ \\
Trygonometryczna (st. 2) & 0,1 & $1,218\cdot 10^{-1} \sin(x_1) + 6,692\cdot 10^{-1} \sin(x_2)\dots$ & $4,877\cdot 10^1$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $4,426\cdot 10^{-1} \sin(x_1) - 6,188\cdot 10^{-2} \cos(x_1)\dots$ & $4,828\cdot 10^1$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $4,426\cdot 10^{-1} \sin(x_1) - 6,188\cdot 10^{-2} \cos(x_1)\dots$ & $3,542\cdot

...

Liniowa & 1e-05 & $-1,962\cdot 10^{-2} + 6,663\cdot 10^{-3} x_1\dots$ & $4,125\cdot 10^{-1}$ \\
Liniowa & 0,0001 & $-1,962\cdot 10^{-2} + 6,663\cdot 10^{-3} x_1\dots$ & $4,125\cdot 10^{-1}$ \\
Liniowa & 0,001 & $-1,962\cdot 10^{-2} + 6,663\cdot 10^{-3} x_1\dots$ & $2,413\cdot 10^{-1}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_4$ & $E_4$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $-1,420\cdot 10^{-3} \sin(x_1) + 1,478\cdot 10^{-3} \sin(x_2)\dots$ & $1,219\cdot 10^{-1}$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $-1,832\cdot 10^{-2} \sin(x_2) - 2,431\cdot 10^{-2} \cos(x_2)\dots$ & $9,300$ \\
Trygonometryczna (st. 2) & 0,1 & $1,493 \sin(x_4) - 2,438\cdot 10^{-1} \sin(2 x_4)$ & $8,369\cdot 10^{-2}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $3,059\cdot 10^{-3} \cos(x_1) - 4,760\cdot 10^{-3} \cos(x_2)\dots$ & $2,836\cdot 10^{-2}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $8,938\cdot 10^{-1} \sin(x_4) - 5,049\cdot 10^{-1} \cos(x_4)\dots$ & $5,032\cdot 10^{-2}$ \\
Tr

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: over

Liniowa & 1e-05 & $-2,746\cdot 10^{-3} + 2,748\cdot 10^{-4} x_1\dots$ & $1,018\cdot 10^{-3}$ \\
Liniowa & 0,0001 & $-2,746\cdot 10^{-3} + 2,748\cdot 10^{-4} x_1\dots$ & $1,018\cdot 10^{-3}$ \\
Liniowa & 0,001 & $-1,928\cdot 10^{-3} + 1,724\cdot 10^{-3} x_2\dots$ & $3,866\cdot 10^{-4}$ \\



Biblioteka funkcji & Próg & $\Dot{x}_5$ & $E_5$ \\
\hline
Trygonometryczna (st. 2) & 0,001 & $1,949\cdot 10^{-2} \sin(x_1) - 2,415\cdot 10^{-3} \sin(x_2)\dots$ & $1,708$ \\
Trygonometryczna (st. 2) & 0,010000000000000002 & $1,132\cdot 10^{-1} \sin(x_2) + 1,291\cdot 10^{-1} \cos(x_2)\dots$ & $1,016\cdot 10^1$ \\
Trygonometryczna (st. 2) & 0,1 & $4,639\cdot 10^{-1} \sin(x_5) + 4,605\cdot 10^{-1} \cos(u)\dots$ & $8,397\cdot 10^{-1}$ \\
\hline
Trygonometryczna (st. 1) & 0,001 & $2,241\cdot 10^{-3} \sin(x_1) + 3,493\cdot 10^{-3} \cos(x_1)\dots$ & $5,342\cdot 10^{-1}$ \\
Trygonometryczna (st. 1) & 0,010000000000000002 & $7,951\cdot 10^{-1} \sin(x_5) - 6,184\cdot 10^{-1} \cos(x_5)\dots$ & $3,084\cdot 10^{-

c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: overflow encountered in reduce
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
c:\Users\ASamek.ERP\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysindy\utils\axes.py:83: RuntimeWarning: over

Liniowa & 1e-05 & $-2,263\cdot 10^{-5} - 6,125\cdot 10^{-4} x_1\dots$ & $9,006\cdot 10^{-2}$ \\
Liniowa & 0,0001 & $-6,125\cdot 10^{-4} x_1 + 2,184\cdot 10^{-3} x_2\dots$ & $9,006\cdot 10^{-2}$ \\
Liniowa & 0,001 & $-4,070\cdot 10^{-3} + 4,746\cdot 10^{-3} x_2\dots$ & $1,475\cdot 10^{-1}$ \\





In [ ]:
plt.plot(t, x_val[:, 0])
plt.plot(t, x_val[:, 1])
plt.plot(t, x_val[:, 2])
plt.plot(t, x_val[:, 3])
plt.plot(t, x_val[:, 4])
plt.grid()
plt.legend(["Zdrowe CD4+", "Zainfekowane CD4+", "Prekursory LTC", "Pomocniczo-niezależne LTC", "Pomocniczo-zależne LTC"])
plt.xlim(0, max(t))
plt.xlabel("Czas [dni]")
ax1 = plt.gca()
ax2 = ax1.twinx()
ax2.spines['right'].set_color('green')
ax2.yaxis.label.set_color('green')
ax2.tick_params(axis='y', colors='green')
ax2.plot(t, u_val(t), 'g--', alpha=0.4)
ax2.set_ylabel("Sterowanie")
ax2.set_ylim(-0.4, 0.4)
ax1.set_ylabel("Liczba osobników")
ax2.set_ylabel("Sterowanie")
plt.show()